Так как сказали ускорится и до завтра сделать оставив некоторые недочеты некая предзащита нам надо ускорится и игратся в тестововй ветке уже нет времени увы 

## Необходимые библиотеки

In [2]:
import pandas as pd
import numpy as np
import ast
import re
import torch
from collections import Counter
# from iterstrat.ml_stratifiers import MultilabelStratifiedKFold Не совсем подходит для моих представлений и целей
from skmultilearn.model_selection import iterative_train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
from sklearn.metrics import classification_report, f1_score, hamming_loss, accuracy_score

c:\Users\josha\OneDrive\Documents\GitHub\4_intensiv_4\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Просмотр изначальных данных

In [3]:
dt = pd.read_csv("original.csv")
dt

,id,rating,comment,Нравится скорость отработки заявок,Нравится качество выполнения заявки,Нравится качество работы сотрудников,Понравилось выполнение заявки,Вопрос решен
0,2945792,5,спасибо,NaN,NaN,NaN,NaN,NaN
1,3234340,5,спасибо!,NaN,NaN,NaN,NaN,NaN
2,3380332,5,Отлично,NaN,NaN,NaN,NaN,NaN
3,3381812,5,Благодарю за оперативное решение проблемы !,NaN,NaN,NaN,NaN,NaN
4,3461991,5,Прекрасный специалист! Побольше таких,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2368,3507812,5,"долго, но приятно",NaN,NaN,NaN,NaN,NaN
2369,3478532,5,16.03.2024 г ребята установили кондиционер. ра...,NaN,NaN,NaN,NaN,NaN
2370,3296403,5,два дня мусоропровод. сегодня с утра тоже,NaN,NaN,NaN,NaN,NaN
2371,3165822,5,отрегулировать дверь замки . для должного откр...,NaN,NaN,NaN,NaN,NaN


#### У нас 2373 неразмеченных данных и 5 категорий для разметки


## Уже размеченные данные

#### Я разметил наши данные через label-studio

In [4]:
df = pd.read_csv("marked_up.csv")
df

,annotation_id,annotator,comment,created_at,id,lead_time,rating,sentiment,updated_at,Вопрос решен,Нравится качество выполнения заявки,Нравится качество работы сотрудников,Нравится скорость отработки заявок,Понравилось выполнение заявки
0,16,2,спасибо,2025-05-08T12:32:06.806278Z,2945792,49.604,5,Вопрос решен,2025-05-08T12:44:22.535929Z,NaN,NaN,NaN,NaN,NaN
1,17,2,спасибо!,2025-05-08T12:44:25.556719Z,3234340,2.058,5,Вопрос решен,2025-05-08T12:44:25.556719Z,NaN,NaN,NaN,NaN,NaN
2,18,2,Отлично,2025-05-08T12:44:46.429049Z,3380332,19.677,5,"{""choices"":[""Понравилось выполнение заявки"",""В...",2025-05-08T12:44:46.429049Z,NaN,NaN,NaN,NaN,NaN
3,19,2,Благодарю за оперативное решение проблемы !,2025-05-08T12:45:09.100228Z,3381812,21.395,5,"{""choices"":[""Нравится скорость отработки заяво...",2025-05-08T12:45:09.100228Z,NaN,NaN,NaN,NaN,NaN
4,20,2,Прекрасный специалист! Побольше таких,2025-05-08T12:45:23.515860Z,3461991,13.272,5,Нравится качество работы сотрудников,2025-05-08T12:45:23.515860Z,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2363,2379,2,"долго, но приятно",2025-05-10T20:12:27.250167Z,3507812,2.124,5,Вопрос решен,2025-05-10T20:12:27.251168Z,NaN,NaN,NaN,NaN,NaN
2364,2380,2,16.03.2024 г ребята установили кондиционер. ра...,2025-05-10T20:12:37.420217Z,3478532,9.359,5,"{""choices"":[""Нравится скорость отработки заяво...",2025-05-10T20:12:37.420217Z,NaN,NaN,NaN,NaN,NaN
2365,2381,2,два дня мусоропровод. сегодня с утра тоже,2025-05-10T20:12:47.220202Z,3296403,8.697,5,Вопрос решен,2025-05-10T20:12:47.220202Z,NaN,NaN,NaN,NaN,NaN
2366,2382,2,отрегулировать дверь замки . для должного откр...,2025-05-10T20:12:52.522679Z,3165822,4.581,5,ни к одному из перечисленных,2025-05-10T20:12:52.522679Z,NaN,NaN,NaN,NaN,NaN


#### это размеченные наши данные гдето потерялось 5 коментариев :(

## Анализ и доработка данных

#### Ну тут очевидно что нам не нужно большинство колонок

In [5]:
df = df.drop(['id' ,'created_at', 'updated_at', 'annotation_id', 'annotator', 'lead_time'], axis=1)
# Смотрим или мы все верно удалили ничего не забыли и ничего лишнего не зацепили(так у нас id не удалился а почему)авхахвп нето вывел ладно
df

,comment,rating,sentiment,Вопрос решен,Нравится качество выполнения заявки,Нравится качество работы сотрудников,Нравится скорость отработки заявок,Понравилось выполнение заявки
0,спасибо,5,Вопрос решен,NaN,NaN,NaN,NaN,NaN
1,спасибо!,5,Вопрос решен,NaN,NaN,NaN,NaN,NaN
2,Отлично,5,"{""choices"":[""Понравилось выполнение заявки"",""В...",NaN,NaN,NaN,NaN,NaN
3,Благодарю за оперативное решение проблемы !,5,"{""choices"":[""Нравится скорость отработки заяво...",NaN,NaN,NaN,NaN,NaN
4,Прекрасный специалист! Побольше таких,5,Нравится качество работы сотрудников,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2363,"долго, но приятно",5,Вопрос решен,NaN,NaN,NaN,NaN,NaN
2364,16.03.2024 г ребята установили кондиционер. ра...,5,"{""choices"":[""Нравится скорость отработки заяво...",NaN,NaN,NaN,NaN,NaN
2365,два дня мусоропровод. сегодня с утра тоже,5,Вопрос решен,NaN,NaN,NaN,NaN,NaN
2366,отрегулировать дверь замки . для должного откр...,5,ни к одному из перечисленных,NaN,NaN,NaN,NaN,NaN


#### Далее надо добавить еще 1 колонку так ка она добавилась в ходе разметки из-за не решонных вопросовы или негатиувных коментариев

In [6]:
df['ни к одному из перечисленных'] = 0
df

,comment,rating,sentiment,Вопрос решен,Нравится качество выполнения заявки,Нравится качество работы сотрудников,Нравится скорость отработки заявок,Понравилось выполнение заявки,ни к одному из перечисленных
0,спасибо,5,Вопрос решен,NaN,NaN,NaN,NaN,NaN,0
1,спасибо!,5,Вопрос решен,NaN,NaN,NaN,NaN,NaN,0
2,Отлично,5,"{""choices"":[""Понравилось выполнение заявки"",""В...",NaN,NaN,NaN,NaN,NaN,0
3,Благодарю за оперативное решение проблемы !,5,"{""choices"":[""Нравится скорость отработки заяво...",NaN,NaN,NaN,NaN,NaN,0
4,Прекрасный специалист! Побольше таких,5,Нравится качество работы сотрудников,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...
2363,"долго, но приятно",5,Вопрос решен,NaN,NaN,NaN,NaN,NaN,0
2364,16.03.2024 г ребята установили кондиционер. ра...,5,"{""choices"":[""Нравится скорость отработки заяво...",NaN,NaN,NaN,NaN,NaN,0
2365,два дня мусоропровод. сегодня с утра тоже,5,Вопрос решен,NaN,NaN,NaN,NaN,NaN,0
2366,отрегулировать дверь замки . для должного откр...,5,ни к одному из перечисленных,NaN,NaN,NaN,NaN,NaN,0


#### Мы добавили колонку теперь перенесем разметку по классам из колонки Sentiment и удалим колонку

In [7]:

stolbiki = [
    'Вопрос решен',
    'Нравится качество выполнения заявки',
    'Нравится качество работы сотрудников',
    'Нравится скорость отработки заявок',
    'Понравилось выполнение заявки',
    'ни к одному из перечисленных'
]

for idx, row in df.iterrows():
    try:
        sentiment = row['sentiment']
        if isinstance(sentiment, str) and '"choices"' in sentiment:
            data = ast.literal_eval(sentiment)
            for label in data.get("choices", []):
                if label in stolbiki:
                    df.at[idx, label] = 1
        elif sentiment in stolbiki:
            df.at[idx, sentiment] = 1
    except Exception as e:
        print(f"Ошибка в строке {idx}: {e}")

#### Посмотим или перенеслось и переименуем колонку 'ни к одному из перечисленных' на более каректное имя 

In [8]:
df = df.rename(columns={'ни к одному из перечисленных': 'Проблемы'})
colum = ['Вопрос решен',
    'Нравится качество выполнения заявки',
    'Нравится качество работы сотрудников',
    'Нравится скорость отработки заявок',
    'Понравилось выполнение заявки',
    'Проблемы']
df[colum] = df[colum].fillna(0)
summa = df[colum].sum()
summa
df = df.drop(['sentiment'], axis=1)


#### Мы посмотрели количество данных  в каждом классов и заполнили пустые значения 0

#### Теперь посмотрик какие слова встречаются чаще в каждом из классов

In [9]:
# def propusk():
#     print("\n"*2)
# для 'Вопрос решен'
klass_bopr = df[df['Вопрос решен'] != 0]
klass_bopr = klass_bopr.drop(['Нравится качество выполнения заявки',
    'Нравится качество работы сотрудников',
    'Нравится скорость отработки заявок',
    'Понравилось выполнение заявки',
    'Проблемы'], axis=1)
klass_bopr['comment'] = klass_bopr['comment'].str.lower()
klass_bopr['comment'] = klass_bopr['comment'].apply(lambda x: re.sub(r"[^а-яёa-z\s]", "", x))
all_words = []
for comment in klass_bopr['comment']:
    words = comment.split()
    all_words.extend(words)
word_counts = Counter(all_words)
top_words = word_counts.most_common(10)
print("Топ-10 самых частых слов в классе 'Вопрос решен'")
for word, count in top_words:
    print(f"{word}: {count} раз")


Топ-10 самых частых слов в классе 'Вопрос решен'
спасибо: 1319 раз
за: 380 раз
быстро: 354 раз
и: 340 раз
большое: 271 раз
все: 270 раз
оперативно: 258 раз
очень: 170 раз
оперативность: 134 раз
в: 131 раз


In [10]:
# для 'Нравится качество выполнения заявки'

klass_bopr = df[df['Нравится качество выполнения заявки'] != 0]

klass_bopr = klass_bopr.drop(['Вопрос решен',
    'Нравится качество работы сотрудников',
    'Нравится скорость отработки заявок',
    'Понравилось выполнение заявки',
    'Проблемы'], axis=1)

klass_bopr['comment'] = klass_bopr['comment'].str.lower()
klass_bopr['comment'] = klass_bopr['comment'].apply(lambda x: re.sub(r"[^а-яёa-z\s]", "", x))
all_words = []
for comment in klass_bopr['comment']:
    words = comment.split()
    all_words.extend(words)
word_counts = Counter(all_words)
top_words = word_counts.most_common(10)
print("Топ-10 самых частых слов в классе 'Нравится качество выполнения заявки'")
for word, count in top_words:
    print(f"{word}: {count} раз")

Топ-10 самых частых слов в классе 'Нравится качество выполнения заявки'
и: 100 раз
качественно: 85 раз
быстро: 81 раз
спасибо: 78 раз
все: 38 раз
за: 29 раз
оперативно: 26 раз
очень: 23 раз
большое: 23 раз
работу: 20 раз


In [11]:
# для Нравится качество работы сотрудников'

klass_bopr = df[df['Нравится качество работы сотрудников'] != 0]

klass_bopr = klass_bopr.drop(['Вопрос решен',
    'Нравится качество выполнения заявки',
    'Нравится скорость отработки заявок',
    'Понравилось выполнение заявки',
    'Проблемы'], axis=1)

klass_bopr['comment'] = klass_bopr['comment'].str.lower()
klass_bopr['comment'] = klass_bopr['comment'].apply(lambda x: re.sub(r"[^а-яёa-z\s]", "", x))
all_words = []
for comment in klass_bopr['comment']:
    words = comment.split()
    all_words.extend(words)
word_counts = Counter(all_words)
top_words = word_counts.most_common(10)
print("Топ-10 самых частых слов в классе 'Нравится качество работы сотрудников'")
for word, count in top_words:
    print(f"{word}: {count} раз")

Топ-10 самых частых слов в классе 'Нравится качество работы сотрудников'
спасибо: 233 раз
и: 136 раз
все: 96 раз
быстро: 88 раз
мастер: 84 раз
за: 82 раз
очень: 64 раз
большое: 61 раз
оперативно: 49 раз
в: 49 раз


In [12]:
# для 'Нравится скорость отработки заявок'
klass_bopr = df[df['Нравится скорость отработки заявок'] != 0]

klass_bopr = klass_bopr.drop(['Вопрос решен',

'Нравится качество выполнения заявки',
    'Нравится качество работы сотрудников',
    'Понравилось выполнение заявки',
    'Проблемы'], axis=1)

klass_bopr['comment'] = klass_bopr['comment'].str.lower()
klass_bopr['comment'] = klass_bopr['comment'].apply(lambda x: re.sub(r"[^а-яёa-z\s]", "", x))
all_words = []
for comment in klass_bopr['comment']:
    words = comment.split()
    all_words.extend(words)
word_counts = Counter(all_words)
top_words = word_counts.most_common(10)
print("Топ-10 самых частых слов в классе 'Нравится скорость отработки заявок'")
for word, count in top_words:
    print(f"{word}: {count} раз")

Топ-10 самых частых слов в классе 'Нравится скорость отработки заявок'
спасибо: 464 раз
быстро: 336 раз
оперативно: 253 раз
за: 251 раз
и: 224 раз
оперативность: 135 раз
очень: 119 раз
все: 119 раз
большое: 92 раз
качественно: 76 раз


In [13]:
# для класса 'Понравилось выполнение заявки'
klass_bopr = df[df['Понравилось выполнение заявки'] != 0]

klass_bopr = klass_bopr.drop(['Вопрос решен',

'Нравится качество выполнения заявки',
    'Нравится качество работы сотрудников',
    'Нравится скорость отработки заявок',
    'Проблемы']
, axis=1)

klass_bopr['comment'] = klass_bopr['comment'].str.lower()
klass_bopr['comment'] = klass_bopr['comment'].apply(lambda x: re.sub(r"[^а-яёa-z\s]", "", x))
all_words = []
for comment in klass_bopr['comment']:
    words = comment.split()
    all_words.extend(words)
word_counts = Counter(all_words)
top_words = word_counts.most_common(10)
print("Топ-10 самых частых слов в классе 'Понравилось выполнение заявки'")
for word, count in top_words:
    print(f"{word}: {count} раз")

Топ-10 самых частых слов в классе 'Понравилось выполнение заявки'
спасибо: 206 раз
и: 114 раз
отлично: 103 раз
все: 102 раз
быстро: 85 раз
за: 65 раз
большое: 52 раз
всё: 51 раз
очень: 43 раз
оперативно: 41 раз


In [14]:
# для класса 'Проблемы'
klass_bopr = df[df['Проблемы'] != 0]

klass_bopr = klass_bopr.drop(['Вопрос решен',

'Нравится качество выполнения заявки',
    'Нравится качество работы сотрудников',
    'Нравится скорость отработки заявок',
    'Понравилось выполнение заявки']
, axis=1)

klass_bopr['comment'] = klass_bopr['comment'].str.lower()
klass_bopr['comment'] = klass_bopr['comment'].apply(lambda x: re.sub(r"[^а-яёa-z\s]", "", x))
all_words = []
for comment in klass_bopr['comment']:
    words = comment.split()
    all_words.extend(words)
word_counts = Counter(all_words)
top_words = word_counts.most_common(10)
print("Топ-10 самых частых слов в классе 'Проблемы'")
for word, count in top_words:
    print(f"{word}: {count} раз")

Топ-10 самых частых слов в классе 'Проблемы'
не: 143 раз
в: 94 раз
и: 76 раз
на: 49 раз
что: 38 раз
за: 32 раз
по: 26 раз
с: 26 раз
так: 25 раз
нет: 24 раз


##### суде по всему людям при плохо выполненной работе или не решонном вопросе любят использовать предлоги, союзы и местоимения 

## Очистка данных


#### Удаление стоп-слов и лемматизация мы делать не будем так как у нас они могут нести смысловую нагрузку 

#### удалим лишние пробелы 

In [15]:

df['comment'] = df['comment'].str.replace(r'\s+', ' ', regex=True)
df

,comment,rating,Вопрос решен,Нравится качество выполнения заявки,Нравится качество работы сотрудников,Нравится скорость отработки заявок,Понравилось выполнение заявки,Проблемы
0,спасибо,5,1.0,0.0,0.0,0.0,0.0,0
1,спасибо!,5,1.0,0.0,0.0,0.0,0.0,0
2,Отлично,5,1.0,0.0,0.0,0.0,1.0,0
3,Благодарю за оперативное решение проблемы !,5,1.0,0.0,0.0,1.0,0.0,0
4,Прекрасный специалист! Побольше таких,5,0.0,0.0,1.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...
2363,"долго, но приятно",5,1.0,0.0,0.0,0.0,0.0,0
2364,16.03.2024 г ребята установили кондиционер. ра...,5,1.0,1.0,1.0,1.0,1.0,1
2365,два дня мусоропровод. сегодня с утра тоже,5,1.0,0.0,0.0,0.0,0.0,0
2366,отрегулировать дверь замки . для должного откр...,5,0.0,0.0,0.0,0.0,0.0,1


## Stratified( или просто процентное уравнивание классов)


#### Буду использовать scikit-multilearn так как насколько я понял при делении клсаов через iterstrat.ml_stratifiers не дает возможности делить данные 80 на 20 и мне посоветовали scikit-multilearn

In [16]:
cm = df["comment"].tolist()
lb = df[['Вопрос решен',
    'Нравится качество выполнения заявки',
    'Нравится качество работы сотрудников',
    'Нравится скорость отработки заявок',
    'Понравилось выполнение заявки',
    'Проблемы']].values

# mlskf = MultilabelStratifiedKFold(n_splits=2, shuffle=True, random_state=42)
# for train_idx, test_idx in mlskf.split(cm, lb):
#     X_train, X_test = [cm[i] for i in train_idx], [cm[i] for i in test_idx]
#     y_train, y_test = lb[train_idx], lb[test_idx]

texts_array = np.array(cm).reshape(-1, 1)

X_train, y_train, X_test, y_test = iterative_train_test_split(
    texts_array,
    lb,
    test_size=0.2
)


X_train = X_train.reshape(-1).tolist()
X_test = X_test.reshape(-1).tolist()

train_label_percent = y_train.sum(axis=0) / y_train.sum() * 100
test_label_percent = y_test.sum(axis=0) / y_test.sum() * 100

print("\nВариант 2: Процент меток класса")
print("Train:", np.round(train_label_percent, 2), "%")
print("Test:", np.round(test_label_percent, 2), "%")


Вариант 2: Процент меток класса
Train: [50.22  3.66  9.84 19.79 10.31  6.17] %
Test: [50.18  3.66  9.8  19.83 10.39  6.14] %


#### ну в процентном соотношении схоже

## Пока целековая моделька пока я до конца не разобрался


In [17]:
print(f"CUDA доступен: {torch.cuda.is_available()}")
print(f"Количество GPU: {torch.cuda.device_count()}")
print(f"Название GPU: {torch.cuda.get_device_name(0)}")

CUDA доступен: True
Количество GPU: 1
Название GPU: NVIDIA GeForce RTX 4070


RuBERT

In [18]:


MODEL_NAME = "DeepPavlov/rubert-base-cased"


tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)


class CustomDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(
            texts, padding=True, truncation=True, max_length=512, return_tensors="pt"
        )
        self.labels = torch.tensor(labels, dtype=torch.float)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': self.labels[idx]
        }

    def __len__(self):
        return len(self.labels)

TEXTS_TRAIN = X_train
Y_TRAIN = y_train
TEXTS_TEST = X_test
Y_TEST = y_test


train_dataset = CustomDataset(TEXTS_TRAIN, Y_TRAIN)
test_dataset = CustomDataset(TEXTS_TEST, Y_TEST)


model = BertForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=Y_TRAIN.shape[1],
    problem_type="multi_label_classification"
)


training_args = TrainingArguments(
    output_dir='./rubert_results',
    per_device_train_batch_size=7,               
    gradient_accumulation_steps=2,                
    num_train_epochs=4,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs_rubert',
    logging_steps=50,
    fp16=True,                                  
    dataloader_num_workers=0,                     
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)


trainer.train()


predictions = trainer.predict(test_dataset)
y_pred_logits = predictions.predictions
y_pred = (torch.sigmoid(torch.tensor(y_pred_logits)) > 0.5).int().numpy()


print("\n🔎 Classification Report:")
print(classification_report(Y_TEST, y_pred, digits=4))

print("\n📊 Дополнительные метрики:")
print(f"Micro F1-score:     {f1_score(Y_TEST, y_pred, average='micro'):.4f}")
print(f"Macro F1-score:     {f1_score(Y_TEST, y_pred, average='macro'):.4f}")
print(f"Weighted F1-score:  {f1_score(Y_TEST, y_pred, average='weighted'):.4f}")
print(f"Hamming loss:       {hamming_loss(Y_TEST, y_pred):.4f}")
print(f"Subset accuracy:    {accuracy_score(Y_TEST, y_pred):.4f} (все метки совпали)")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.241200,0.188540
2,0.126400,0.163882
3,0.066700,0.162750



🔎 Classification Report:
              precision    recall  f1-score   support

           0     0.9541    0.9788    0.9663       425
           1     0.8571    0.7742    0.8136        31
           2     0.8261    0.9157    0.8686        83
           3     0.9360    0.9583    0.9471       168
           4     0.7000    0.5568    0.6203        88
           5     0.8372    0.6923    0.7579        52

   micro avg     0.9061    0.8996    0.9028       847
   macro avg     0.8518    0.8127    0.8289       847
weighted avg     0.9009    0.8996    0.8986       847
 samples avg     0.9145    0.9128    0.9012       847


📊 Дополнительные метрики:
Micro F1-score:     0.9028
Macro F1-score:     0.8289
Weighted F1-score:  0.8986
Hamming loss:       0.0577
Subset accuracy:    0.7300 (все метки совпали)


c:\Users\josha\OneDrive\Documents\GitHub\4_intensiv_4\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


RuRoBERTa  очень долго и очень плохо


In [19]:


class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.encodings = tokenizer(
            texts, padding=True, truncation=True, max_length=512, return_tensors="pt"
        )
        self.labels = torch.tensor(labels, dtype=torch.float)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': self.labels[idx]
        }

    def __len__(self):
        return len(self.labels)

if __name__ == "__main__":
    MODEL_NAME = "sberbank-ai/ruRoberta-large"


    tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)

    train_dataset = CustomDataset(X_train, y_train, tokenizer)
    test_dataset = CustomDataset(X_test, y_test, tokenizer)

    model = RobertaForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=y_train.shape[1],
        problem_type="multi_label_classification"
    )

    training_args = TrainingArguments(
        output_dir='./ruroberta_results',
        per_device_train_batch_size=8,
        num_train_epochs=4,
        eval_strategy="epoch",
        save_strategy="epoch",
        logging_dir='./logs_ruroberta',
        logging_steps=50,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        fp16=False,
        dataloader_num_workers=0  
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
    )

    trainer.train()

    predictions = trainer.predict(test_dataset)
    y_pred_logits = predictions.predictions
    y_pred = (torch.sigmoid(torch.tensor(y_pred_logits)) > 0.5).int().numpy()

    print("🔎 Classification Report:")
    print(classification_report(y_test, y_pred, digits=4))

    print("\n📊 Дополнительные метрики:")
    print(f"Micro F1-score:     {f1_score(y_test, y_pred, average='micro'):.4f}")
    print(f"Macro F1-score:     {f1_score(y_test, y_pred, average='macro'):.4f}")
    print(f"Weighted F1-score:  {f1_score(y_test, y_pred, average='weighted'):.4f}")
    print(f"Hamming loss:       {hamming_loss(y_test, y_pred):.4f}")
    print(f"Subset accuracy:    {accuracy_score(y_test, y_pred):.4f} (все метки совпали)")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at sberbank-ai/ruRoberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.438400,0.431134
2,0.418500,0.423759
3,0.412600,0.427658
4,0.412000,0.434308


🔎 Classification Report:
              precision    recall  f1-score   support

           0     0.8966    1.0000    0.9455       425
           1     0.0000    0.0000    0.0000        31
           2     0.0000    0.0000    0.0000        83
           3     0.0000    0.0000    0.0000       168
           4     0.0000    0.0000    0.0000        88
           5     0.0000    0.0000    0.0000        52

   micro avg     0.8966    0.5018    0.6435       847
   macro avg     0.1494    0.1667    0.1576       847
weighted avg     0.4499    0.5018    0.4744       847
 samples avg     0.8966    0.5874    0.6787       847


📊 Дополнительные метрики:
Micro F1-score:     0.6435
Macro F1-score:     0.1576
Weighted F1-score:  0.4744
Hamming loss:       0.1656
Subset accuracy:    0.3418 (все метки совпали)


c:\Users\josha\OneDrive\Documents\GitHub\4_intensiv_4\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\josha\OneDrive\Documents\GitHub\4_intensiv_4\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [20]:
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.encodings = tokenizer(
            texts, padding=True, truncation=True, max_length=512, return_tensors="pt"
        )
        self.labels = torch.tensor(labels, dtype=torch.float)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': self.labels[idx]
        }

    def __len__(self):
        return len(self.labels)

if __name__ == "__main__":
    MODEL_NAME = "bert-base-multilingual-cased"

    tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

    train_dataset = CustomDataset(X_train, y_train, tokenizer)
    test_dataset = CustomDataset(X_test, y_test, tokenizer)

    model = BertForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=y_train.shape[1],
        problem_type="multi_label_classification"
    )

    training_args = TrainingArguments(
        output_dir='./multi_bert_results',
        per_device_train_batch_size=16,
        num_train_epochs=4,
        eval_strategy="epoch",
        save_strategy="epoch",
        logging_dir='./logs_multi_bert',
        logging_steps=50,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        fp16=False,
        dataloader_num_workers=0 
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
    )

    trainer.train()

    predictions = trainer.predict(test_dataset)
    y_pred_logits = predictions.predictions
    y_pred = (torch.sigmoid(torch.tensor(y_pred_logits)) > 0.5).int().numpy()

    print("🔎 Classification Report:")
    print(classification_report(y_test, y_pred, digits=4))

    print("\n📊 Дополнительные метрики:")
    print(f"Micro F1-score:     {f1_score(y_test, y_pred, average='micro'):.4f}")
    print(f"Macro F1-score:     {f1_score(y_test, y_pred, average='macro'):.4f}")
    print(f"Weighted F1-score:  {f1_score(y_test, y_pred, average='weighted'):.4f}")
    print(f"Hamming loss:       {hamming_loss(y_test, y_pred):.4f}")
    print(f"Subset accuracy:    {accuracy_score(y_test, y_pred):.4f} (все метки совпали)")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.247900,0.224609
2,0.170800,0.201651
3,0.129100,0.178141
4,0.091500,0.181756


🔎 Classification Report:
              precision    recall  f1-score   support

           0     0.9518    0.9765    0.9640       425
           1     0.8846    0.7419    0.8070        31
           2     0.8902    0.8795    0.8848        83
           3     0.9573    0.9345    0.9458       168
           4     0.6374    0.6591    0.6480        88
           5     0.8095    0.6538    0.7234        52

   micro avg     0.9037    0.8973    0.9005       847
   macro avg     0.8551    0.8076    0.8288       847
weighted avg     0.9030    0.8973    0.8993       847
 samples avg     0.9122    0.9076    0.8970       847


📊 Дополнительные метрики:
Micro F1-score:     0.9005
Macro F1-score:     0.8288
Weighted F1-score:  0.8993
Hamming loss:       0.0591
Subset accuracy:    0.7384 (все метки совпали)


c:\Users\josha\OneDrive\Documents\GitHub\4_intensiv_4\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
